In [1]:
cd imagery

/home/stevetyl/Documents/UAVtoDESCARTES/imagery


# Uploading UAV Imagery to Descartes
* Step 1: Install descarteslabs into your environment and other required libraries. 
* Step 2: In a terminal, within your environment, authenticate using: "descarteslabs auth login". This will prompt you to log in using your DescartesLabs credentials and provide a token, available through a link in the terminal or on the online portal. 
* Step 3: Run through the code below:

In [7]:
#Import DescartesLabs Libraries
import descarteslabs as dl
import descarteslabs.workflows as wf
import descarteslabs.catalog
from descarteslabs.catalog import Band, DataType, Product, Resolution, ResolutionUnit, SpectralBand, Image
from descarteslabs.scenes import Scene

#Import Other Libraries
import rasterio
import gdal
import numpy as np
import re

#Create New Product
def create_new_product(product_name):
    #Create Catalog Product
    product_id = str(product_name)
    new_product = Product(id=product_id)
    new_product.name = str(product_name)
    new_product.readers = ["org:pivotbio"]
    new_product.writers = ["org:pivotbio"]
    new_product.save()
    print(new_product.state)

    for band_index, color in enumerate(["red", "green", "blue", "nir", "rededge"]):
        band = SpectralBand(name=color, product = new_product)
        band.data_type = DataType.FLOAT32
        if color == "red":
            band.display_range = (0, 6.5) 
        
        elif color == "green":
            band.display_range = (0, 4)
        
        elif color == "blue":
            band.display_range = (0, 4)
        
        elif color == "nir": 
            band.display_range = (0, 15)
        
        elif color == "rededge":
            band.display_range = (0, 50)
        
        band.data_range = (0, 10000)    
        band.resolution = Resolution(unit = ResolutionUnit.METERS, value = 0.027938946951286157)
        band.band_index = band_index
        band.nodata = -10000
        band.save()
        print(band.id)
    catalog_product = new_product
    return catalog_product

#Actual Upload of Imagery
def upload_uav_descartes(filename, product):
    #Determine Date
    p = re.compile(r'\d\d\d\d\d\d\d\d')
    number = p.findall(filename)

    datelist = []
    j = 0
    for i in number[0]:
        datelist.append(i)
        if j == 3 or j == 5:
            datelist.append("-")   
        j += 1
    
    date = "".join(datelist)
       
    image = Image(product = product, name = filename)
    image.acquired = str(date)

    upload = image.upload(filename)
    upload.wait_for_completion()
    return upload.status

<br>

## Initiate Product Name and Files to Upload

In [24]:
product_name = "2020-SENTERA-UAV" #input()
filename = "20-ZEAMX-US301-MO03-MOAR_20200807_6x_UAV.tif"
file_list = ["20-ZEAMX-US301-MO03-MOAR_20200807_6x_UAV.tif", "20-ZEAMX-US301-MO03-MOAR_20200707_6x_UAV.tif", "20-ZEAMX-US301-MO03-MOAR_20200818_6x_UAV.tif"]



#Some notes: 
#make it so you can select the files from a window (optionally) 
#make it so you can provide the files as arguments on the command line
#make it so you can provide the product name in the command line.
#make it so you can type the product name in a window. 
#
#

#What about the data warehouse? 

#What about AWS? Download then upload? - maybe a good question for Descartes. 

<br>

## Check if Catalog Product Already Exists

In [25]:
#Check if Catalog Product Exists
catalog_product = Product.get("pivotbio:"+ str(product_name))
try: 
    catalog_product.state #Get current state (either saved/unsaved if exists. Else error. )
    print("Product is in catalog. Proceed with image upload.")
    x = 0
except:
    print("Product is not in catalog. Proceed with product creation.")
    x = 1

Product is in catalog. Proceed with image upload.


<br>

## Create Catalog Product
This will create a new product with the product name provided above with read/write permissions available to all accounts in the Pivot Bio organization. 

In [18]:
#Conditionally create or update a product. 
if x == 0:
    catalog_product = Product.get("pivotbio:" + str(product_name)) #This grabs a product that currently exists. 
    print("id:", catalog_product.id, "\nstatus:", catalog_product.state)
if x == 1:
    create_new_product(product_name) #This returns the catalog product name. 

id: pivotbio:2020-SENTERA-UAV 
status: DocumentState.SAVED


<br>

## Upload Image(s) 
This will upload an image to our product from the list of file paths provided above. With some testing, a 500 mb image takes approximately 13 minutes to upload. If uploading multiple files, take this upload time into consideration. 

In [25]:
#Upload each image file in the list provided above. 
for i in file_list:
    upload_uav_descartes(i, catalog_product)

KeyboardInterrupt: 

In [344]:
#Open Imagery & Grab Data
filename = "LC81390452014295LGN00_B1.tiff"

with rasterio.open(filename, 'r') as ds:
    arr = ds.read()  # read all raster values
    meta = ds.meta
    trans = ds.transform
    print(ds.meta)
    
g = gdal.Open (filename)
geo_transform = g.GetGeoTransform ()
geom = g.GetProjection()

pixelSizeX = geo_transform[1]
pixelSizeY =-geo_transform[5]

proj = osr.SpatialReference(wkt = geom)
EPSG = proj.GetAttrValue('AUTHORITY', 1)

info = gdal.Info(filename, format = "json")
geometry = {"type": "Polygon","coordinates": info['wgs84Extent']['coordinates']}

{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': None, 'width': 7621, 'height': 7791, 'count': 1, 'crs': CRS.from_epsg(32645), 'transform': Affine(30.0, 0.0, 381885.0,
       0.0, -30.0, 2512815.0)}


In [345]:
import rasterio

AWS_link = 'https://landsat-pds.s3.amazonaws.com/L8/139/045/LC81390452014295LGN00/LC81390452014295LGN00_B1.TIF'

src = rasterio.open(AWS_link)
src.meta

bounds = src.bounds
left= bounds.left
bottom = bounds.bottom
right = bounds.right
top = bounds.top

In [346]:
print(bounds)

BoundingBox(left=381885.0, bottom=2279085.0, right=610515.0, top=2512815.0)


In [349]:
from descarteslabs.catalog import Product, Image, StorageState
product_2 = Product(id="pivotbio:aws_product", name = "aws product")
product_2.save()
#status = product_2.delete_related_objects()


In [352]:
product_2.state

<DocumentState.SAVED: 'saved'>

In [367]:
#Create Bands
band = SpectralBand(name="band1", product=product_2)
band.data_type = DataType.UINT16
band.data_range = (np.amin(arr[arr != nodatavalue][0]), np.max(arr[0]))
band.display_range = (np.amin(arr[arr != nodatavalue][0]), np.max(arr[0]))
band.resolution = Resolution(unit=ResolutionUnit.METERS, value = geo_transform[1])
band.band_index = 0
band.nodata= None

band.save()

In [370]:
image = Image(product = product_2, name = "test_aws_image") 
image.acquired = "2020-08-07"
image.acquired_end = "2020-08-07"
image.geotrans = geo_transform
image.projection = EPSG

image.geometry = geometry
image.files = [File(href="https://landsat-pds.s3.amazonaws.com/L8/139/045/LC81390452014295LGN00/LC81390452014295LGN00_B1.TIF")]
image.x_pixels = int(pixelSizeX)
image.y_pixels = int(pixelSizeY) 
image.hash  = 
image.storage_state = StorageState.AVAILABLE
image.save()

BadRequestError: 
    Validation error: index 0, field size_bytes: Required when storage_state='available': files
    Validation error: index 0, field hash: Required when storage_state='available': files
    Validation error: index 0, field href: 'https://landsat-pds.s3.amazonaws.com/L8/139/045/LC81390452014295LGN00/LC81390452014295LGN00_B1.TIF' must start with 'gs://' when storage_state='available': storage_state